In [1]:
import tensorflow as tf
import tensorflow_probability as tfp

2023-02-12 14:15:44.133429: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Define the model
def linear_regression(x, a, b):
  return a * x + b

# Define the prior distributions for the model parameters
a_prior = tfp.distributions.Normal(loc=0., scale=10.)
b_prior = tfp.distributions.Normal(loc=0., scale=10.)

# Define the likelihood distribution
noise = tfp.distributions.Normal(loc=0., scale=1.)

# Generate observed data
num_samples = 1000
x_observed = tfp.distributions.Uniform(0., 1.).sample(num_samples)
y_observed = linear_regression(x_observed, 0.5, 2.0) + noise.sample(num_samples)

2023-02-12 14:15:46.718574: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-12 14:15:48.070154: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10214 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060, pci bus id: 0000:09:00.0, compute capability: 8.6
2023-02-12 14:15:48.070941: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 5591 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2070 SUPER, pci bus id: 0000:05:00.0, compute capability: 7.5


In [3]:
# Define the posterior distributions
def posterior_mean_field(x, a, b):
  a_posterior = tfp.distributions.Normal(loc=a_prior.mean(), scale=a_prior.stddev())
  b_posterior = tfp.distributions.Normal(loc=b_prior.mean(), scale=b_prior.stddev())
  return tfp.distributions.Normal(loc=linear_regression(x, a_posterior, b_posterior),
                                  scale=noise.stddev())

In [4]:
def loss_fn(a, b):
  log_probs = posterior_mean_field(x_observed, a, b).log_prob(y_observed)
  return -tf.reduce_mean(log_probs)


In [7]:
def loss_fn(a, b):
  log_probs = tf.reduce_sum(posterior_mean_field(x_observed, a, b).log_prob(y_observed))
  return -log_probs


In [10]:
def loss_fn(a, b):
  log_probs = tf.map_fn(
      lambda x: posterior_mean_field(x, a, b).log_prob(y_observed[x]),
      tf.range(num_samples),
      dtype=tf.float32
  )
  return -tf.reduce_mean(log_probs)


In [11]:
# Use gradient descent to optimize the parameters
optimizer = tf.optimizers.Adam(0.1)
num_steps = 5000
a_hat, b_hat = 0.0, 0.0
for step in range(num_steps):
  with tf.GradientTape() as tape:
    loss = loss_fn(a_hat, b_hat)
  grads = tape.gradient(loss, [a_hat, b_hat])
  optimizer.apply_gradients(zip(grads, [a_hat, b_hat]))
  if (step % 500 == 0):
    print("Step {}: a = {}, b = {}".format(step, a_hat.numpy(), b_hat.numpy()))

print("Final a: {}, b: {}".format(a_hat.numpy(), b_hat.numpy()))


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


ValueError: in user code:

    File "/tmp/ipykernel_26291/2799401400.py", line 3, in None  *
        lambda x: posterior_mean_field(x, a, b).log_prob(y_observed[x])
    File "/tmp/ipykernel_26291/1906716884.py", line 5, in posterior_mean_field  *
        return tfp.distributions.Normal(loc=linear_regression(x, a_posterior, b_posterior),
    File "/tmp/ipykernel_26291/2502962390.py", line 3, in linear_regression  *
        return a * x + b

    ValueError: TypeError: object of type 'Normal' has no len()
    
